Trains a ResNet on the CIFAR10 dataset.

ResNet v1:
[Deep Residual Learning for Image Recognition
](https://arxiv.org/pdf/1512.03385.pdf)

ResNet v2:
[Identity Mappings in Deep Residual Networks
](https://arxiv.org/pdf/1603.05027.pdf)


Model|n|200-epoch accuracy|Original paper accuracy |sec/epoch GTX1080Ti
:------------|--:|-------:|-----------------------:|---:
ResNet20   v1|  3| 92.16 %|                 91.25 %|35
ResNet32   v1|  5| 92.46 %|                 92.49 %|50
ResNet44   v1|  7| 92.50 %|                 92.83 %|70
ResNet56   v1|  9| 92.71 %|                 93.03 %|90
ResNet110  v1| 18| 92.65 %|            93.39+-.16 %|165
ResNet164  v1| 27|     - %|                 94.07 %|  -
ResNet1001 v1|N/A|     - %|                 92.39 %|  -

&nbsp;

Model|n|200-epoch accuracy|Original paper accuracy |sec/epoch GTX1080Ti
:------------|--:|-------:|-----------------------:|---:
ResNet20   v2|  2|     - %|                     - %|---
ResNet32   v2|N/A| NA    %|            NA         %| NA
ResNet44   v2|N/A| NA    %|            NA         %| NA
ResNet56   v2|  6| 93.01 %|            NA         %|100
ResNet110  v2| 12| 93.15 %|            93.63      %|180
ResNet164  v2| 18|     - %|            94.54      %|  -
ResNet1001 v2|111|     - %|            95.08+-.14 %|  -

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)

/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sunxin/minico

1.13.2
2.2.4


Using TensorFlow backend.


In [2]:
import numpy as np
from keras.datasets import cifar10
from keras import models
from keras import layers
from keras import regularizers
from keras import optimizers
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

In [3]:
# Training parameters
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 200
data_augmentation = True
num_classes = 10

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
n = 3

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


In [4]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [5]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = layers.Conv2D(num_filters,
                         kernel_size=kernel_size,
                         strides=strides,
                         padding='same',
                         kernel_initializer='he_normal',
                         kernel_regularizer=regularizers.l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = layers.BatchNormalization()(x)
        if activation is not None:
            x = layers.Activation(activation)(x)
    else:
        if batch_normalization:
            x = layers.BatchNormalization()(x)
        if activation is not None:
            x = layers.Activation(activation)(x)
        x = conv(x)
    return x

In [6]:
def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = layers.Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = layers.add([x, y])
            x = layers.Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = layers.AveragePooling2D(pool_size=8)(x)
    y = layers.Flatten()(x)
    outputs = layers.Dense(num_classes,
                           activation='softmax',
                           kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = models.Model(inputs=inputs, outputs=outputs)
    return model

In [7]:
if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

Instructions for updating:
Colocations handled automatically by placer.
Learning rate:  0.001
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
_______________

In [8]:
# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

In [9]:
# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch=int(np.ceil(x_train.shape[0] / batch_size)),
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1, workers=4,
                        callbacks=callbacks)

Using real-time data augmentation.
Instructions for updating:
Use tf.cast instead.
Epoch 1/200
Learning rate:  0.001
1563/1563 [==============================] - 50s 32ms/step - loss: 1.5576 - acc: 0.4933 - val_loss: 2.2981 - val_acc: 0.3799

Epoch 00001: val_acc improved from -inf to 0.37990, saving model to /home/sunxin/qhm_experiment/experiments/saved_models/cifar10_ResNet20v1_model.001.h5
Epoch 2/200
Learning rate:  0.001
1563/1563 [==============================] - 45s 29ms/step - loss: 1.1653 - acc: 0.6439 - val_loss: 1.2549 - val_acc: 0.6308

Epoch 00002: val_acc improved from 0.37990 to 0.63080, saving model to /home/sunxin/qhm_experiment/experiments/saved_models/cifar10_ResNet20v1_model.002.h5
Epoch 3/200
Learning rate:  0.001
1563/1563 [==============================] - 46s 29ms/step - loss: 1.0214 - acc: 0.6976 - val_loss: 1.5450 - val_acc: 0.5450

Epoch 00003: val_acc did not improve from 0.63080
Epoch 4/200
Learning rate:  0.001
1563/1563 [==============================] -

Epoch 33/200
Learning rate:  0.001
1563/1563 [==============================] - 45s 29ms/step - loss: 0.5605 - acc: 0.8778 - val_loss: 0.8177 - val_acc: 0.8119

Epoch 00033: val_acc did not improve from 0.84590
Epoch 34/200
Learning rate:  0.001
1563/1563 [==============================] - 46s 29ms/step - loss: 0.5588 - acc: 0.8778 - val_loss: 0.7777 - val_acc: 0.8207

Epoch 00034: val_acc did not improve from 0.84590
Epoch 35/200
Learning rate:  0.001
1563/1563 [==============================] - 45s 29ms/step - loss: 0.5566 - acc: 0.8785 - val_loss: 0.7192 - val_acc: 0.8341

Epoch 00035: val_acc did not improve from 0.84590
Epoch 36/200
Learning rate:  0.001
1563/1563 [==============================] - 45s 29ms/step - loss: 0.5525 - acc: 0.8809 - val_loss: 0.7722 - val_acc: 0.8179

Epoch 00036: val_acc did not improve from 0.84590
Epoch 37/200
Learning rate:  0.001
1563/1563 [==============================] - 45s 29ms/step - loss: 0.5519 - acc: 0.8814 - val_loss: 0.7963 - val_acc: 0.8

1563/1563 [==============================] - 44s 28ms/step - loss: 0.5059 - acc: 0.8964 - val_loss: 0.6534 - val_acc: 0.8625

Epoch 00070: val_acc did not improve from 0.86720
Epoch 71/200
Learning rate:  0.001
1563/1563 [==============================] - 44s 28ms/step - loss: 0.5057 - acc: 0.8985 - val_loss: 0.6971 - val_acc: 0.8383

Epoch 00071: val_acc did not improve from 0.86720
Epoch 72/200
Learning rate:  0.001
1563/1563 [==============================] - 44s 28ms/step - loss: 0.5015 - acc: 0.9000 - val_loss: 0.6743 - val_acc: 0.8510

Epoch 00072: val_acc did not improve from 0.86720
Epoch 73/200
Learning rate:  0.001
1563/1563 [==============================] - 44s 28ms/step - loss: 0.5029 - acc: 0.9007 - val_loss: 0.6632 - val_acc: 0.8520

Epoch 00073: val_acc did not improve from 0.86720
Epoch 74/200
Learning rate:  0.001
1563/1563 [==============================] - 44s 28ms/step - loss: 0.5033 - acc: 0.8982 - val_loss: 0.7449 - val_acc: 0.8327

Epoch 00074: val_acc did not i

Epoch 105/200
Learning rate:  0.0001
1563/1563 [==============================] - 44s 28ms/step - loss: 0.2443 - acc: 0.9677 - val_loss: 0.4543 - val_acc: 0.9142

Epoch 00105: val_acc improved from 0.91160 to 0.91420, saving model to /home/sunxin/qhm_experiment/experiments/saved_models/cifar10_ResNet20v1_model.105.h5
Epoch 106/200
Learning rate:  0.0001
1563/1563 [==============================] - 44s 28ms/step - loss: 0.2426 - acc: 0.9679 - val_loss: 0.4472 - val_acc: 0.9098

Epoch 00106: val_acc did not improve from 0.91420
Epoch 107/200
Learning rate:  0.0001
1563/1563 [==============================] - 44s 28ms/step - loss: 0.2393 - acc: 0.9678 - val_loss: 0.4701 - val_acc: 0.9088

Epoch 00107: val_acc did not improve from 0.91420
Epoch 108/200
Learning rate:  0.0001
1563/1563 [==============================] - 44s 28ms/step - loss: 0.2371 - acc: 0.9683 - val_loss: 0.4478 - val_acc: 0.9113

Epoch 00108: val_acc did not improve from 0.91420
Epoch 109/200
Learning rate:  0.0001
1563/

1563/1563 [==============================] - 44s 28ms/step - loss: 0.1807 - acc: 0.9825 - val_loss: 0.4486 - val_acc: 0.9129

Epoch 00143: val_acc did not improve from 0.91420
Epoch 144/200
Learning rate:  1e-05
1563/1563 [==============================] - 44s 28ms/step - loss: 0.1798 - acc: 0.9829 - val_loss: 0.4485 - val_acc: 0.9120

Epoch 00144: val_acc did not improve from 0.91420
Epoch 145/200
Learning rate:  1e-05
1563/1563 [==============================] - 44s 28ms/step - loss: 0.1806 - acc: 0.9817 - val_loss: 0.4495 - val_acc: 0.9129

Epoch 00145: val_acc did not improve from 0.91420
Epoch 146/200
Learning rate:  1e-05
1563/1563 [==============================] - 44s 28ms/step - loss: 0.1784 - acc: 0.9829 - val_loss: 0.4475 - val_acc: 0.9128

Epoch 00146: val_acc did not improve from 0.91420
Epoch 147/200
Learning rate:  1e-05
1563/1563 [==============================] - 44s 28ms/step - loss: 0.1789 - acc: 0.9826 - val_loss: 0.4483 - val_acc: 0.9137

Epoch 00147: val_acc did n

1563/1563 [==============================] - 44s 28ms/step - loss: 0.1733 - acc: 0.9837 - val_loss: 0.4530 - val_acc: 0.9135

Epoch 00181: val_acc did not improve from 0.91480
Epoch 182/200
Learning rate:  5e-07
1563/1563 [==============================] - 44s 28ms/step - loss: 0.1708 - acc: 0.9850 - val_loss: 0.4523 - val_acc: 0.9137

Epoch 00182: val_acc did not improve from 0.91480
Epoch 183/200
Learning rate:  5e-07
1563/1563 [==============================] - 44s 28ms/step - loss: 0.1718 - acc: 0.9843 - val_loss: 0.4505 - val_acc: 0.9133

Epoch 00183: val_acc did not improve from 0.91480
Epoch 184/200
Learning rate:  5e-07
1563/1563 [==============================] - 44s 28ms/step - loss: 0.1715 - acc: 0.9838 - val_loss: 0.4506 - val_acc: 0.9140

Epoch 00184: val_acc did not improve from 0.91480
Epoch 185/200
Learning rate:  5e-07
1563/1563 [==============================] - 44s 28ms/step - loss: 0.1730 - acc: 0.9840 - val_loss: 0.4522 - val_acc: 0.9131

Epoch 00185: val_acc did n

In [10]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 174us/step
Test loss: 0.45462303552627564
Test accuracy: 0.913
